In [1]:
import pandas as pd
import zipfile
import glob

In [3]:
#Read the zip file that contains multiple csv file
from zipfile import ZipFile
from google.colab import files
!wget 'https://raw.githubusercontent.com/Myloveismilk/vn_stock_price/main/data/archive.zip'
url = '/content/archive.zip'
df = pd.concat(
    [pd.read_csv(ZipFile(url).open(i)) for i in ZipFile(url).namelist()],
    ignore_index=True
)

--2022-10-09 09:04:31--  https://raw.githubusercontent.com/Myloveismilk/vn_stock_price/main/data/archive.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3066402 (2.9M) [application/zip]
Saving to: ‘archive.zip.1’

archive.zip.1       100%[===================>]   2.92M  --.-KB/s    in 0.07s   

2022-10-09 09:04:31 (44.4 MB/s) - ‘archive.zip.1’ saved [3066402/3066402]



In [4]:
#See the first 5 rows of the dataset
df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,TB Base Volume,TB Quote Volume,Ignore
0,2019-07-30 00:00:00.000000000,9509.07,9535.00,9503.07,9517.75,381.293856,2019-07-30 00:29:59.999000064,3.631211e+06,3492,216.528742,2.062189e+06,0
1,2019-07-30 00:30:00.000000000,9517.03,9539.00,9507.00,9532.58,258.514869,2019-07-30 00:59:59.999000064,2.462134e+06,3133,141.887550,1.351182e+06,0
2,2019-07-30 01:00:00.000000000,9533.59,9534.00,9500.00,9500.25,275.797270,2019-07-30 01:29:59.999000064,2.624322e+06,3583,114.238457,1.087402e+06,0
3,2019-07-30 01:30:00.000000000,9500.25,9514.46,9460.71,9465.25,485.218470,2019-07-30 01:59:59.999000064,4.604004e+06,5330,225.681291,2.141459e+06,0
4,2019-07-30 02:00:00.000000000,9465.25,9488.94,9430.01,9460.42,498.669626,2019-07-30 02:29:59.999000064,4.717269e+06,6197,228.535910,2.162163e+06,0


In [5]:
df.columns

Index(['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time',
       'Quote Asset Volume', 'Number of Trades', 'TB Base Volume',
       'TB Quote Volume', 'Ignore'],
      dtype='object')

In [7]:
#Some columns isn't necessary for predicting the bitcoin price, so let's remove it.
df = df.drop(columns= ['Close Time','Quote Asset Volume', 'Number of Trades', 'TB Base Volume','TB Quote Volume', 'Ignore'])

In [8]:
df.columns

Index(['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [15]:
#The names of all columns seem to be not in the right format, so let's change them
df = df.rename(columns= {'Open Time':'Date'})

In [16]:
df.head()

,Date,Open,High,Low,Close,Volume
0,2019-07-30 00:00:00.000000000,9509.07,9535.00,9503.07,9517.75,381.293856
1,2019-07-30 00:30:00.000000000,9517.03,9539.00,9507.00,9532.58,258.514869
2,2019-07-30 01:00:00.000000000,9533.59,9534.00,9500.00,9500.25,275.797270
3,2019-07-30 01:30:00.000000000,9500.25,9514.46,9460.71,9465.25,485.218470
4,2019-07-30 02:00:00.000000000,9465.25,9488.94,9430.01,9460.42,498.669626


In [20]:
#Let's look at the type of each column
df['Date'] = df['Date'].astype('string')
df.dtypes

Date       string
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
dtype: object

In [40]:
#We don't need a specific time in our dataset, just need the date
def remove_time(column):
  temp = 0
  for i in range(len(column)):
    column[i] = column[i][0: 10]
    if temp <= 10:
      print(column[i])
      temp += 1

In [41]:
remove_time(df['Date'])

2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30
2019-07-30


In [42]:
df.head()

,Date,Open,High,Low,Close,Volume
0,2019-07-30,9509.07,9535.00,9503.07,9517.75,381.293856
1,2019-07-30,9517.03,9539.00,9507.00,9532.58,258.514869
2,2019-07-30,9533.59,9534.00,9500.00,9500.25,275.797270
3,2019-07-30,9500.25,9514.46,9460.71,9465.25,485.218470
4,2019-07-30,9465.25,9488.94,9430.01,9460.42,498.669626


In [45]:
df.tail()

,Date,Open,High,Low,Close,Volume
55357,2022-09-27,20294.87,20298.61,20179.46,20194.41,11124.18405
55358,2022-09-27,20195.87,20291.64,20169.52,20269.48,10877.37598
55359,2022-09-27,20269.48,20278.27,20178.01,20183.74,10273.22546
55352,2022-09-27,20099.04,20136.36,19982.39,20048.22,11175.34812
55381,2022-09-27,18950.71,19159.83,18924.64,19103.23,15654.96482


In [46]:
#Change the data type of 'Date' column
df['Date'] = df['Date'].astype('datetime64')

#Set the 'Date' column to index values so it can be easily to plot time series chart.
df = df.set_index('Date')

In [47]:
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2019-07-30,9509.07,9535.00,9503.07,9517.75,381.293856
2019-07-30,9535.84,9554.88,9525.30,9530.43,583.603684
2019-07-30,9528.75,9552.29,9526.96,9541.88,452.309510
2019-07-30,9542.24,9576.83,9537.69,9567.87,510.042282
2019-07-30,9567.87,9577.80,9542.00,9558.25,508.236202


In [49]:
#The dataset now look fine, let's save it for future work.
df.to_csv('bitcoin_price.csv', index = False)